## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-01-16-51 +0000,nyt,Virginia Democrats Propose New Congressional M...,https://www.nytimes.com/2026/02/05/us/politics...
1,2026-02-06-01-12-00 +0000,wsj,Oil Falls Amid Signs of Easing Supply Risks,https://www.wsj.com/finance/commodities-future...
2,2026-02-06-01-09-14 +0000,nypost,‘Vermin infestation’ closed Santa Monica hotsp...,https://nypost.com/2026/02/05/us-news/vermin-i...
3,2026-02-06-01-08-57 +0000,nypost,Extraordinary distraction scam shows how ATM t...,https://nypost.com/2026/02/05/us-news/extraord...
4,2026-02-06-01-05-06 +0000,nypost,Spencer Pratt reveals whistle blowers tipped h...,https://nypost.com/2026/02/05/us-news/spencer-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2511/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
186,trump,30
209,epstein,22
3,new,15
121,ai,14
110,california,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
83,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,91
92,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,74
137,2026-02-05-20-27-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,71
110,2026-02-05-21-49-42 +0000,bbc,Trump's nuclear arms control push with Russia ...,https://www.bbc.com/news/articles/c70l27gy47do...,64
346,2026-02-05-03-27-44 +0000,bbc,Trump says America should move on from Epstein...,https://www.bbc.com/news/articles/cy4gj71z0m0o...,64


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
83,91,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
27,60,2026-02-06-00-25-42 +0000,nypost,Match.com sent Jeffrey Epstein dating profiles...,https://nypost.com/2026/02/05/us-news/doj-file...
53,53,2026-02-05-23-41-25 +0000,nypost,Elite California universities spending up to $...,https://nypost.com/2026/02/05/us-news/elite-ca...
251,52,2026-02-05-14-39-47 +0000,nypost,Rick Caruso considers return to LA mayor’s rac...,https://nypost.com/2026/02/05/us-news/rick-car...
59,46,2026-02-05-23-29-46 +0000,nypost,"Russia, Ukraine exchange 300 prisoners as US p...",https://nypost.com/2026/02/05/world-news/russi...
336,38,2026-02-05-05-11-00 +0000,wsj,Democrats’ drive to reach a deal to put new re...,https://www.wsj.com/politics/policy/congress-e...
115,38,2026-02-05-21-40-00 +0000,startribune,Minneapolis snowboarder Iris Pflum’s early-mor...,https://www.startribune.com/iris-pflum-2026-wi...
240,38,2026-02-05-15-27-00 +0000,wsj,The Bank of England and ECB kept interest rate...,https://www.wsj.com/economy/central-banking/ba...
56,37,2026-02-05-23-37-02 +0000,startribune,"In rare move, this northern Minnesota county i...",https://www.startribune.com/in-rare-move-this-...
14,35,2026-02-06-00-44-49 +0000,bbc,US police believe Savannah Guthrie's missing m...,https://www.bbc.com/news/articles/c2e18wvrx9xo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
